# Building a Drug Response Report

For this notebook, you are going to focus on a single region in the genome, defined as chromosome 22, for all 2,504 samples in the Thousand Genomes dataset. As chromosome 22 was the first chromosome to be sequenced as part of the Human Genome Project, it is your first here as well. 

What small molecules/drugs are most likely to affect a subpopulation of individuals (ancestry, age, etc.) based on their genomic information?

In this query, assume that you have some phenotype data about your population. In this case, also assume that all samples sharing the pattern “NA12” are part of a specific demographic.

In this query, use sampleid as your predicate pushdown. The general steps are:

1. Filter by the samples in your subpopulation
2. Aggregate variant frequencies for the subpopulation-of-interest
3. Join on ClinVar dataset
4. Filter by variants that have been implicated in drug-response
5. Order by highest frequency variants

The raw clinvar data and a parquet version of chromosome 22 of 1000 genomes, partitioned by sample id, are in your data lake. You also have a VCF in your data lake for chromosome 22 of 1000 genomes.

### Import Dependencies

In [14]:
import boto3, os

s3 = boto3.resource('s3')
glue = boto3.client('glue')
cfn = boto3.client('cloudformation')

In [15]:
import sys
!{sys.executable} -m pip install PyAthena

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [16]:
from pyathena import connect
import pandas as pd
from pyathena.pandas.util import as_pandas

### Define Variables

In [17]:
import jmespath

session = boto3.session.Session()
region = session.region_name
print(region)

project_name = os.environ.get('RESOURCE_PREFIX')
database_name = project_name.lower()
work_group_name = project_name.lower() + '-' + region
print(project_name)
print(database_name)
print(work_group_name)

resources = cfn.describe_stacks(StackName='{0}-Pipeline'.format(project_name))
query = 'Stacks[].Outputs[?OutputKey==`DataLakeBucket`].OutputValue'
data_lake_bucket = path = jmespath.search(query, resources)[0][0]
print(data_lake_bucket)


us-west-1
GenomicsAnalysis
genomicsanalysis
genomicsanalysis-us-west-1
genomicsanalysis-pipeline-datalakebucket-1g5hynzwz9kmd


### Create drug response result set

In [18]:
conn = connect(s3_staging_dir='s3://%s/results/drug_response' % data_lake_bucket, region_name=region)
cursor = conn.cursor(work_group=work_group_name)
cursor.execute(
        "SELECT"
        "    count(*)/cast(numsamples AS DOUBLE) AS genotypefrequency "
        "    ,cv.rsid "
        "    ,cv.phenotypelist "
        "    ,sv.chromosome "
        "    ,sv.startposition "
        "    ,sv.endposition "
        "    ,sv.referenceallele "
        "    ,sv.alternateallele "
        "    ,sv.genotype0 "
        "    ,sv.genotype1 "
        "FROM \"" + database_name + "\".onekg_chr22_by_sample sv "
        "CROSS JOIN "
        "    (SELECT count(1) AS numsamples "
        "    FROM "
        "        (SELECT DISTINCT sampleid "
        "        FROM \"" + database_name + "\".onekg_chr22_by_sample "
        "        WHERE sampleid LIKE 'NA12%')) "
        "JOIN \"" + database_name + "\".clinvar cv "
        "ON sv.chromosome = cv.chromosome "
        "    AND sv.startposition = cv.start - 1 "
        "    AND sv.endposition = cv.stop "
        "    AND sv.referenceallele = cv.referenceallele "
        "    AND sv.alternateallele = cv.alternateallele "
        "WHERE assembly='GRCh37' "
        "    AND cv.clinicalsignificance LIKE '%response%' "
        "    AND sampleid LIKE 'NA12%' "
        "GROUP BY  sv.chromosome "
        "          ,sv.startposition "
        "          ,sv.endposition "
        "          ,sv.referenceallele "
        "          ,sv.alternateallele "
        "          ,sv.genotype0 "
        "          ,sv.genotype1 "
        "          ,cv.clinicalsignificance "
        "          ,cv.phenotypelist "
        "          ,cv.rsid "
        "          ,numsamples "
        "ORDER BY  genotypefrequency DESC LIMIT 50 ")

df = as_pandas(cursor)
df

,genotypefrequency,rsid,phenotypelist,chromosome,startposition,endposition,referenceallele,alternateallele,genotype0,genotype1
0,0.553846,4680,CATECHOL-O-METHYLTRANSFERASE POLYMORPHISM;meth...,22,19951270,19951271,G,A,0,1
1,0.507692,2298383,caffeine response - Toxicity/ADR,22,24825510,24825511,C,T,0,1
2,0.369231,3892097,"Debrisoquine, poor metabolism of;amitriptyline...",22,42524946,42524947,C,T,0,1
3,0.353846,2298383,caffeine response - Toxicity/ADR,22,24825510,24825511,C,T,1,1
4,0.292308,738409,"Fatty liver disease, nonalcoholic 1;Susceptibi...",22,44324726,44324727,C,G,0,1
5,0.276923,13306278,Selective serotonin reuptake inhibitors respon...,22,19929026,19929027,C,T,0,1
6,0.153846,4680,CATECHOL-O-METHYLTRANSFERASE POLYMORPHISM;meth...,22,19951270,19951271,G,A,1,1
7,0.046154,738409,"Fatty liver disease, nonalcoholic 1;Susceptibi...",22,44324726,44324727,C,G,1,1
8,0.046154,13306278,Selective serotonin reuptake inhibitors respon...,22,19929026,19929027,C,T,1,1
9,0.030769,3892097,"Debrisoquine, poor metabolism of;amitriptyline...",22,42524946,42524947,C,T,1,1


### Query annotation dataset

In [19]:
conn = connect(s3_staging_dir='s3://%s/results/annotation/clinvar' % data_lake_bucket,region_name=region)
cursor = conn.cursor(work_group=work_group_name)
cursor.execute('SELECT * FROM "%s"."clinvar" limit 10' % database_name)
df = as_pandas(cursor)
df

,alleleid,type,name,geneid,genesymbol,hgnc_id,clinicalsignificance,clinsigsimple,lastevaluated,rsid,...,referenceallele,alternateallele,cytogenetic,reviewstatus,numbersubmitters,guidelines,testedingtr,otherids,submittercategories,variationid
0,15041,indel,NM_014855.3(AP5Z1):c.80_83delinsTGCTGTAAACTGTA...,9907,AP5Z1,HGNC:22197,Pathogenic,1,"Jun 29, 2010",397704705,...,GGAT,TGCTGTAAACTGTAACTGTAAA,7p22.1,no assertion criteria provided,1,None,N,OMIM Allelic Variant:613653.0001,1,2
1,15041,indel,NM_014855.3(AP5Z1):c.80_83delinsTGCTGTAAACTGTA...,9907,AP5Z1,HGNC:22197,Pathogenic,1,"Jun 29, 2010",397704705,...,GGAT,TGCTGTAAACTGTAACTGTAAA,7p22.1,no assertion criteria provided,1,None,N,OMIM Allelic Variant:613653.0001,1,2
2,15042,deletion,NM_014855.3(AP5Z1):c.1413_1426del (p.Leu473fs),9907,AP5Z1,HGNC:22197,Pathogenic,1,"Jun 29, 2010",397704709,...,GGACCTGCCCTGCT,-,7p22.1,no assertion criteria provided,1,None,N,OMIM Allelic Variant:613653.0002,1,3
3,15042,deletion,NM_014855.3(AP5Z1):c.1413_1426del (p.Leu473fs),9907,AP5Z1,HGNC:22197,Pathogenic,1,"Jun 29, 2010",397704709,...,GGACCTGCCCTGCT,-,7p22.1,no assertion criteria provided,1,None,N,OMIM Allelic Variant:613653.0002,1,3
4,15043,single nucleotide variant,NM_014630.3(ZNF592):c.3136G>A (p.Gly1046Arg),9640,ZNF592,HGNC:28986,Uncertain significance,0,"Jun 29, 2015",150829393,...,G,A,15q25,no assertion criteria provided,1,None,N,"OMIM Allelic Variant:613624.0001,UniProtKB (pr...",1,4
5,15043,single nucleotide variant,NM_014630.3(ZNF592):c.3136G>A (p.Gly1046Arg),9640,ZNF592,HGNC:28986,Uncertain significance,0,"Jun 29, 2015",150829393,...,G,A,15q25.3,no assertion criteria provided,1,None,N,"OMIM Allelic Variant:613624.0001,UniProtKB (pr...",1,4
6,15044,single nucleotide variant,NM_017547.4(FOXRED1):c.694C>T (p.Gln232Ter),55572,FOXRED1,HGNC:26927,Pathogenic,1,"Dec 07, 2017",267606829,...,C,T,11q24,"criteria provided, single submitter",2,None,N,OMIM Allelic Variant:613622.0001,3,5
7,15044,single nucleotide variant,NM_017547.4(FOXRED1):c.694C>T (p.Gln232Ter),55572,FOXRED1,HGNC:26927,Pathogenic,1,"Dec 07, 2017",267606829,...,C,T,11q24.2,"criteria provided, single submitter",2,None,N,OMIM Allelic Variant:613622.0001,3,5
8,15045,single nucleotide variant,NM_017547.4(FOXRED1):c.1289A>G (p.Asn430Ser),55572,FOXRED1,HGNC:26927,Pathogenic,1,"Oct 01, 2010",267606830,...,A,G,11q24,no assertion criteria provided,1,None,N,"OMIM Allelic Variant:613622.0002,UniProtKB (pr...",1,6
9,15045,single nucleotide variant,NM_017547.4(FOXRED1):c.1289A>G (p.Asn430Ser),55572,FOXRED1,HGNC:26927,Pathogenic,1,"Oct 01, 2010",267606830,...,A,G,11q24.2,no assertion criteria provided,1,None,N,"OMIM Allelic Variant:613622.0002,UniProtKB (pr...",1,6


### Query cohort dataset

In [20]:
conn = connect(s3_staging_dir='s3://%s/results/variants/' % data_lake_bucket,region_name=region)
cursor = conn.cursor(work_group=work_group_name)
cursor.execute("SELECT * FROM \"" + database_name + "\".onekg_chr22_by_sample WHERE sampleid LIKE 'NA12%' limit 10")
df = as_pandas(cursor)
df


,alternateallele,chromosome,endposition,genotype0,genotype1,referenceallele,sampleid,startposition
0,G,22,32726515,0,1,C,NA12283,32726514
1,G,22,32726515,1,1,C,NA12750,32726514
2,G,22,32726515,1,1,C,NA12872,32726514
3,T,22,32726587,1,1,A,NA12046,32726586
4,T,22,32726587,1,1,A,NA12383,32726586
5,T,22,32726587,1,1,A,NA12777,32726586
6,G,22,32726606,1,1,A,NA12283,32726605
7,G,22,32726606,1,1,A,NA12750,32726605
8,G,22,32726606,1,1,A,NA12872,32726605
9,G,22,32726649,0,1,GGT,NA12814,32726646


### Query individual variant dataset

In [21]:
conn = connect(s3_staging_dir='s3://%s/results/vcf/' % data_lake_bucket,region_name=region)
cursor = conn.cursor(work_group=work_group_name)
cursor.execute("SELECT * FROM \"" + database_name + "\".vcf limit 10")
df = as_pandas(cursor)
df


,v.contig,v.start,v.ref,v.altalleles,va.rsid,va.qual,va.filters,va.info.end,va.info.blockavg_min30p3a,va.info.snvhpol,va.info.cigar,va.info.ru,va.info.refrep,va.info.idrep,va.info.mq
0,chr1,14776,G,"[{ref=G, alt=A}]",None,0.0,[LowGQX],None,None,3,None,None,None,None,7.0
1,chr1,14794,G,"[{ref=G, alt=C}]",None,0.0,"[LowDepth, LowGQX]",None,None,5,None,None,None,None,6.0
2,chr1,16014,C,"[{ref=C, alt=T}]",None,5.0,"[LowDepth, LowGQX]",None,None,2,None,None,None,None,7.0
3,chr1,16674,G,"[{ref=G, alt=T}]",None,1.0,[LowGQX],None,None,3,None,None,None,None,7.0
4,chr1,16866,G,"[{ref=G, alt=T}]",None,1.0,"[LowDepth, LowGQX]",None,None,2,None,None,None,None,4.0
5,chr1,16957,G,"[{ref=G, alt=T}]",None,3.0,"[LowDepth, LowGQX]",None,None,2,None,None,None,None,3.0
6,chr1,17004,C,"[{ref=C, alt=G}]",None,3.0,"[LowDepth, LowGQX]",None,None,3,None,None,None,None,4.0
7,chr1,17006,A,"[{ref=A, alt=G}]",None,3.0,"[LowDepth, LowGQX]",None,None,2,None,None,None,None,4.0
8,chr1,17259,T,"[{ref=T, alt=C}]",None,0.0,[LowGQX],None,None,2,None,None,None,None,15.0
9,chr1,17365,C,"[{ref=C, alt=G}]",None,17.0,[LowGQX],None,None,4,None,None,None,None,14.0
